In [1]:
import pandas as pd
import numpy as np

In [2]:
tips=pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [3]:
tips['tip_pct']=tips['tip']/tips['total_bill']

In [4]:
def top(df,n=5,column='tip_pct'):  #특정 컬럼에서 가장 큰 값을 가지는 로우들을 선택
    return df.sort_values(by=column)[-n:]

top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [5]:
tips.groupby('smoker').apply(top) # top 함수가 나뉘어진 df의 각 부분에 모두 적용

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [6]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill') #추가적인 인자를 받는다.

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

## 1. 그룹 색인 생략하기

In [7]:
tips.groupby('smoker',group_keys=False).apply(top) #group_keys=False를 넘겨서 그룹 색인 생략

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## 2.변위치 분석과 버킷 분석

In [8]:
frame=pd.DataFrame({'data1':np.random.randn(1000),
                   'data2':np.random.randn(1000)})

qualtiles=pd.cut(frame.data1,4)
qualtiles[:10]

0       (1.521, 3.117]
1     (-0.0755, 1.521]
2    (-1.672, -0.0755]
3     (-0.0755, 1.521]
4    (-1.672, -0.0755]
5    (-1.672, -0.0755]
6       (1.521, 3.117]
7    (-1.672, -0.0755]
8    (-1.672, -0.0755]
9    (-1.672, -0.0755]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.275, -1.672] < (-1.672, -0.0755] < (-0.0755, 1.521] < (1.521, 3.117]]

In [9]:
# cut에서 반환된 Categorical 객체는 바로 groupby로 넘길 수 있다.
def get_stats(group):
    return {'min':group.min(),'max':group.max(),
           'count':group.count(),'mean':group.mean()}

grouped=frame.data2.groupby(qualtiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.275, -1.672]",-2.107465,3.121893,62.0,0.133999
"(-1.672, -0.0755]",-2.716656,2.941815,417.0,-0.019522
"(-0.0755, 1.521]",-3.327279,2.589663,452.0,0.050782
"(1.521, 3.117]",-1.880366,1.927103,69.0,0.131093


In [10]:
# qcut을 사용
grouping=pd.qcut(frame.data1,10,labels=False)
grouped=frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.107465,3.121893,100.0,0.088969
1,-2.716656,2.546636,100.0,0.043663
2,-2.578819,1.872193,100.0,-0.139212
3,-2.325547,2.357448,100.0,0.099063
4,-2.223364,2.941815,100.0,-0.095749
5,-2.342762,2.324354,100.0,-0.058607
6,-2.915410,2.340678,100.0,0.056032
7,-3.327279,2.589663,100.0,-0.083706
8,-2.979467,2.479340,100.0,0.228390


## 3. 그룹에 따른 값으로 결측치 채우기

In [11]:
# 누락된 값을 평균값으로 대체하기
s=pd.Series(np.random.randn(6))
s[::2]=np.nan
s


0         NaN
1    0.487264
2         NaN
3   -1.796246
4         NaN
5    0.103465
dtype: float64

In [12]:
s.fillna(s.mean())

0   -0.401839
1    0.487264
2   -0.401839
3   -1.796246
4   -0.401839
5    0.103465
dtype: float64

In [13]:
# 그룹별로 다르게 값을 채워넣기
states=['Ohio','New York','Vermont','Florida',
      'Oregon','Nevada','California','Idaho']
group_key=['East']*4+['West']*4  # 리스트에 있는 네벌의 원소를 이어붙인다.
data=pd.Series(np.random.randn(8),index=states)
data

Ohio          2.188786
New York      0.770739
Vermont       0.464312
Florida      -1.674465
Oregon        0.506168
Nevada        0.602130
California    0.620443
Idaho        -0.144457
dtype: float64

In [14]:
# 임의로 결측값 지정
data[['Vermont','Nevada','Idaho']]=np.nan
data

Ohio          2.188786
New York      0.770739
Vermont            NaN
Florida      -1.674465
Oregon        0.506168
Nevada             NaN
California    0.620443
Idaho              NaN
dtype: float64

In [15]:
# group_key값으로 그룹화하기
data.groupby(group_key).mean()

East    0.428353
West    0.563305
dtype: float64

In [16]:
# 누락된 값을 그룹의 평균값으로 채우기
fill_mean=lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          2.188786
New York      0.770739
Vermont       0.428353
Florida      -1.674465
Oregon        0.506168
Nevada        0.563305
California    0.620443
Idaho         0.563305
dtype: float64

In [17]:
# 미리 정의된 다른 값을 채워 넣는다.
fill_values={'East':0.5,'West':-1}
fill_func=lambda g:g.fillna(fill_values[g.name]) #각 그룹은 내부적으로 name이라는 속성을 가지고있다.
data.groupby(group_key).apply(fill_func)

Ohio          2.188786
New York      0.770739
Vermont       0.500000
Florida      -1.674465
Oregon        0.506168
Nevada       -1.000000
California    0.620443
Idaho        -1.000000
dtype: float64

## 4. 랜덤 표본과 순열

In [18]:
# 트럼프 카드덱 만들기
suits=['H','S','C','D']
card_val=(list(range(1,11))+[10]*3)*4
base_names=['A']+list(range(2,11))+['J','K','Q']
cards=[]
for suit in ['H','S','C','D']:
    cards.extend(str(num)+suit for num in base_names)
deck=pd.Series(card_val,index=cards)

In [19]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [20]:
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)

QC    10
KH    10
QH    10
5D     5
9S     9
dtype: int64

In [21]:
get_suit=lambda card:card[-1]
deck.groupby(get_suit).apply(draw,n=2)

C  AC      1
   6C      6
D  3D      3
   QD     10
H  QH     10
   KH     10
S  10S    10
   3S      3
dtype: int64

In [22]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

3C      3
4C      4
10D    10
AD      1
7H      7
2H      2
6S      6
5S      5
dtype: int64

## 5.그룹 가중 평균과 상관관계

In [23]:
df=pd.DataFrame({'category':['a','a','a','a',
                            'b','b','b','b'],
                'data':np.random.randn(8),
                'weights':np.random.rand(8)})
df

,category,data,weights
0,a,-0.598991,0.790969
1,a,-1.530252,0.308436
2,a,0.060837,0.362224
3,a,-1.596669,0.172959
4,b,-0.803531,0.248400
5,b,-2.222021,0.501801
6,b,-1.142436,0.851810
7,b,-1.404105,0.270305


In [24]:
grouped=df.groupby('category')
get_wavg=lambda g:np.average(g['data'],weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.734063
b   -1.424591
dtype: float64

In [25]:
# 주식데이터 예제
close_px=pd.read_csv('stock_px.csv',parse_dates=True,index_col=0)

In [26]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [27]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [28]:
# spx컬럼과 다른 컬럼들의 상관관계
spx_corr=lambda x:x.corrwith(x['SPX'])

In [29]:
rets=close_px.pct_change().dropna() #close_px의 퍼센트 변화율 구하기

In [30]:
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [31]:
get_year=lambda x:x.year
by_year=rets.groupby(get_year) #연도별 퍼센트 변화율 구하기
by_year.apply(spx_corr)  #상관관계

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [32]:
# 두 컬럼간의 상관관계
by_year.apply(lambda g:g["AAPL"].corr(g["MSFT"]))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 6. 그룹상의 선형회귀

In [33]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y=data[yvar]
    X=data[xvars]
    X['intercept']=1
    result=sm.OLS(Y,X).fit()
    return result.params


In [34]:
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514
